<a href="https://colab.research.google.com/github/gox6/buzzer/blob/main/buzzer_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project setup

In [6]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [111]:
import duckdb
from functools import reduce
import lightgbm as lgb
import numpy as np
import optuna
import pandas as pd
import polars as pl
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from statistics import mean

In [8]:
# Getting data from Google Drive file
url='https://drive.google.com/uc?id=1FrSdVAw_nmouSCE7VZ54-_HfM_C9DIC6'
df_orig = pl.read_csv(url)

In [9]:
# Renaming columns using snake_case
df = df_orig[:]
to_snake_case = lambda x: '_'.join(x.lower().split(' '))
df.columns = list(map(to_snake_case, df_orig.columns))

## Helicopter view EDA

In [10]:
df.head(5)

lead_number,lead_origin,lead_source,do_not_email,do_not_call,converted,totalvisits,total_time_spent_on_website,page_views_per_visit,last_activity,country,specialization,how_did_you_hear_about_x_education,what_is_your_current_occupation,what_matters_most_to_you_in_choosing_a_course,search,magazine,newspaper_article,x_education_forums,newspaper,digital_advertisement,through_recommendations,receive_more_updates_about_our_courses,tags,lead_quality,update_me_on_supply_chain_content,get_updates_on_dm_content,lead_profile,city,asymmetrique_activity_index,asymmetrique_profile_index,asymmetrique_activity_score,asymmetrique_profile_score,i_agree_to_pay_the_amount_through_cheque,a_free_copy_of_mastering_the_interview,last_notable_activity
i64,str,str,str,str,i64,f64,i64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str,str,str
660737,"""API""","""Olark Chat""","""No""","""No""",0,0.0,0,0.0,"""Page Visited o…","""India""","""Others""",null,"""Unemployed""","""Better Career …","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Interested in …","""Low in Relevan…","""No""","""No""",null,null,"""02.Medium""","""02.Medium""",15.0,15.0,"""No""","""No""","""Modified"""
660728,"""API""","""Organic Search…","""No""","""No""",0,5.0,674,2.5,"""Email Opened""","""India""","""Others""",null,"""Unemployed""","""Better Career …","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Ringing""","""Not Sure""","""No""","""No""",null,null,"""02.Medium""","""02.Medium""",15.0,15.0,"""No""","""No""","""Email Opened"""
660727,"""Landing Page S…","""Direct Traffic…","""No""","""No""",1,2.0,1532,2.0,"""Email Opened""","""India""","""Business Admin…",null,"""Student""","""Better Career …","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Will revert af…","""Might be""","""No""","""No""","""Potential Lead…","""Mumbai""","""02.Medium""","""01.High""",14.0,20.0,"""No""","""Yes""","""Email Opened"""
660719,"""Landing Page S…","""Direct Traffic…","""No""","""No""",0,1.0,305,1.0,"""Unreachable""","""India""","""Media and Adve…","""Word Of Mouth""","""Unemployed""","""Better Career …","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Ringing""","""Not Sure""","""No""","""No""",null,"""Mumbai""","""02.Medium""","""01.High""",13.0,17.0,"""No""","""No""","""Modified"""
660681,"""Landing Page S…","""Google""","""No""","""No""",1,2.0,1428,1.0,"""Converted to L…","""India""","""Others""","""Other""","""Unemployed""","""Better Career …","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Will revert af…","""Might be""","""No""","""No""",null,"""Mumbai""","""02.Medium""","""01.High""",15.0,18.0,"""No""","""No""","""Modified"""


In [11]:
df.describe()

describe,lead_number,lead_origin,lead_source,do_not_email,do_not_call,converted,totalvisits,total_time_spent_on_website,page_views_per_visit,last_activity,country,specialization,how_did_you_hear_about_x_education,what_is_your_current_occupation,what_matters_most_to_you_in_choosing_a_course,search,magazine,newspaper_article,x_education_forums,newspaper,digital_advertisement,through_recommendations,receive_more_updates_about_our_courses,tags,lead_quality,update_me_on_supply_chain_content,get_updates_on_dm_content,lead_profile,city,asymmetrique_activity_index,asymmetrique_profile_index,asymmetrique_activity_score,asymmetrique_profile_score,i_agree_to_pay_the_amount_through_cheque,a_free_copy_of_mastering_the_interview,last_notable_activity
str,f64,str,str,str,str,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str,str,str
"""count""",9240.0,"""9240""","""9240""","""9240""","""9240""",9240.0,9240.0,9240.0,9240.0,"""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""",9240.0,9240.0,"""9240""","""9240""","""9240"""
"""null_count""",0.0,"""0""","""36""","""0""","""0""",0.0,137.0,0.0,137.0,"""103""","""0""","""0""","""7250""","""0""","""2709""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""6855""","""3669""","""4218""","""4218""",4218.0,4218.0,"""0""","""0""","""0"""
"""mean""",617188.435606,null,null,null,null,0.38539,3.445238,487.698268,2.36282,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,14.306252,16.344883,null,null,null
"""std""",23405.995698,null,null,null,null,0.486714,4.854853,548.021466,2.161418,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.386694,1.811395,null,null,null
"""min""",579533.0,"""API""","""Click2call""","""No""","""No""",0.0,0.0,0.0,0.0,"""Approached upf…","""Asia/Pacific R…","""Banking, Inves…","""Advertisements…","""Businessman""","""Better Career …","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Already a stud…","""High in Releva…","""No""","""No""","""Dual Specializ…","""Mumbai""","""01.High""","""01.High""",7.0,11.0,"""No""","""No""","""Approached upf…"
"""max""",660737.0,"""Quick Add Form…","""youtubechannel…","""Yes""","""Yes""",1.0,251.0,2272.0,55.0,"""Visited Booth …","""unknown""","""Travel and Tou…","""Word Of Mouth""","""Working Profes…","""Other""","""Yes""","""No""","""Yes""","""Yes""","""Yes""","""Yes""","""Yes""","""No""","""wrong number g…","""Worst""","""No""","""No""","""Student of Som…","""Tier II Cities…","""03.Low""","""03.Low""",18.0,20.0,"""No""","""Yes""","""View in browse…"
"""median""",615479.0,null,null,null,null,0.0,3.0,248.0,2.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,14.0,16.0,null,null,null
"""25%""",596485.0,null,null,null,null,0.0,1.0,12.0,1.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,14.0,15.0,null,null,null
"""75%""",637388.0,null,null,null,null,1.0,5.0,936.0,3.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,15.0,18.0,null,null,null


In [12]:
# Types of variables

for column, type_ in zip(df.columns, df.dtypes):
  print(f"{column:<50}{type_}")

lead_number                                       Int64
lead_origin                                       Utf8
lead_source                                       Utf8
do_not_email                                      Utf8
do_not_call                                       Utf8
converted                                         Int64
totalvisits                                       Float64
total_time_spent_on_website                       Int64
page_views_per_visit                              Float64
last_activity                                     Utf8
country                                           Utf8
specialization                                    Utf8
how_did_you_hear_about_x_education                Utf8
what_is_your_current_occupation                   Utf8
what_matters_most_to_you_in_choosing_a_course     Utf8
search                                            Utf8
magazine                                          Utf8
newspaper_article                                 Utf8
x

## Prep for modelling

In [13]:
# Assigning roles to variables

df = df.with_columns(pl.col('total_time_spent_on_website').cast(pl.Float64))

cat_vars = df.select([pl.col(pl.Utf8)]).columns
num_vars = df.select([pl.col(pl.Float64)]).columns
skip_vars = ['lead_number']
target = 'converted'

# Check that all variables were assigned the role
assert len(df.columns) == len(cat_vars + num_vars + skip_vars + [target])


In [14]:
X = df.select(cat_vars + num_vars)
y = df.select(pl.col(target)).to_numpy().reshape(-1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)

In [15]:
cat_enc = OrdinalEncoder()
cat_enc.fit(X.select(cat_vars))

std_scaler =  StandardScaler()
std_scaler.fit(X_train.select(num_vars))

def preprocess(X, encoders):
    X_partial_encodings = map(lambda enc: enc.transform(X.select(enc.feature_names_in_)), encoders)
    return reduce(lambda x, y: np.concatenate([x, y], axis=1), X_partial_encodings)

X_train_pp = preprocess(X_train, encoders=[cat_enc, std_scaler])
X_test_pp = preprocess(X_test, encoders=[cat_enc, std_scaler])

In [58]:
def train_lgb(X_train, y_train,
              X_test, y_test,
              X_val=None, y_val=None,
              params = {'objective': 'binary', 'n_estimators': 5000},
              early_stopping_patience = 100,
              ):
  
    std_scaler =  StandardScaler()
    std_scaler.fit(X_train.select(num_vars))

    X_train_pp = preprocess(X_train, encoders=[cat_enc, std_scaler])
    X_test_pp = preprocess(X_test, encoders=[cat_enc, std_scaler])

    if X_val is not None:
        X_val_pp = preprocess(X_val, encoders=[cat_enc, std_scaler])
        eval_set = [(X_val_pp, y_val)]
        eval_names = 'val'
    else:
        eval_set = [(X_test_pp, y_test)]
        eval_names = 'test'


    
    clf = lgb.sklearn.LGBMClassifier(**params)

    
    callbacks = [lgb.early_stopping(early_stopping_patience, verbose=False), lgb.log_evaluation(period=0)]

    clf.fit(X_train_pp, y_train,
            eval_set =  eval_set,
            eval_names = eval_names,
            callbacks=callbacks,
            feature_name = cat_vars + num_vars,
            #categorical_feature=list(range(len(cat_vars))),
            )

    return clf.score(X_test_pp, y_test), std_scaler, clf

In [56]:
def cross_validate(X_train, y_train, n_folds=5, params={'objective': 'binary', 'n_estimators': 5000}):

    kf = KFold(n_splits=n_folds, shuffle=True, random_state=0)

    metrics = []

    for train_train_indices, train_val_test_indices in kf.split(X_train):

        X_train_train = X_train[train_train_indices]
        y_train_train = y_train[train_train_indices]

        X_train_val_test = X_train[train_val_test_indices]
        y_train_val_test = y_train[train_val_test_indices]
        
        X_train_val, X_train_test, y_train_val, y_train_test = train_test_split(X_train_val_test, y_train_val_test, test_size=0.5, random_state=0)

        # metric, _, _ = train_lgb(X_train_train, y_train_train,
        #                    X_train_test, y_train_test,
        #                    X_val=X_train_val, y_val=y_train_val,
        #                    params = params)
        
        metric, _, _ = train_lgb(X_train_train, y_train_train,
                           X_train_val_test, y_train_val_test,

                           params = params)
        metrics.append(metric)

    return mean(metrics)
        
  

In [138]:
def objective(trial):

    # Define hyperparameters
    params = {
        'objective': 'binary',
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.2),
        'num_leaves': trial.suggest_int('num_leaves', 10, 128),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 0.1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 0.1),
        'n_estimators': 5000,
    }
    
    # Train model
    metric = cross_validate(X_train, y_train, n_folds=5, params=params)
    
    # Return cross-validated accuracy
    return metric

default_params = {
                  'objective': 'binary',
                  'learning_rate': 0.1,
                  'num_leaves': 31,
                  'min_child_samples': 20,
                  'scale_pos_weight': 1,
                  'reg_alpha': 0,
                  'reg_lambda': 0,
                  'n_estimators': 5000,
                  }
  
study = optuna.create_study(direction='maximize', study_name="Buzzer: Propensity to buy", sampler=optuna.samplers.TPESampler())
study.enqueue_trial(default_params)

study.optimize(objective, n_trials=200, gc_after_trial=True)

[I 2023-05-08 12:07:09,219] A new study created in memory with name: Buzzer: Propensity to buy
[I 2023-05-08 12:07:11,328] Trial 0 finished with value: 0.9445350834095012 and parameters: {'learning_rate': 0.1, 'num_leaves': 31, 'min_child_samples': 20, 'scale_pos_weight': 1.0, 'reg_alpha': 0.0, 'reg_lambda': 0.0}. Best is trial 0 with value: 0.9445350834095012.
[I 2023-05-08 12:07:17,104] Trial 1 finished with value: 0.9389875029849558 and parameters: {'learning_rate': 0.104793105391747, 'num_leaves': 54, 'min_child_samples': 34, 'scale_pos_weight': 3.635216010692467, 'reg_alpha': 0.0628882803365363, 'reg_lambda': 0.07303490033767736}. Best is trial 0 with value: 0.9445350834095012.
[I 2023-05-08 12:07:20,291] Trial 2 finished with value: 0.9397995024616165 and parameters: {'learning_rate': 0.1415344536474595, 'num_leaves': 120, 'min_child_samples': 40, 'scale_pos_weight': 2.886175655244708, 'reg_alpha': 0.03823982294644845, 'reg_lambda': 0.08502910165164267}. Best is trial 0 with valu

In [141]:
best_params_after_200_trials_with_TPE_sampler = {'learning_rate': 0.006788716196826271,
                                                  'num_leaves': 44,
                                                  'min_child_samples': 45,
                                                  'scale_pos_weight': 1.1848069434581796,
                                                  'reg_alpha': 0.07885543177387411,
                                                  'reg_lambda': 0.002309836099942583}

In [171]:
# Best parameters, with validation set incorporated into training set
best_params = study.best_params

optuna_accuracy, optuna_std_scaler, optuna_clf = train_lgb(X_train, y_train,
                                      X_test, y_test,
                                      params = best_params,
                                      early_stopping_patience = 5000
                                      )
accuracy, std_scaler, clf = train_lgb(X_train, y_train,
                                      X_test, y_test,
                                      params = default_params,
                                      early_stopping_patience = 5000
                                      )

# There is overfitting in hyperparameter optimization: random_state has to be changed for each iteration
print(f"Model accuracy with best params from hyperparameter optimization: {optuna_accuracy}")
print(f"Model accuracy with default params: {accuracy}")

Model accuracy with best params from hyperparameter optimization: 0.9237012987012987
Model accuracy with default params: 0.9534632034632035


In [147]:
X_test_pp = preprocess(X_test, encoders=[cat_enc, std_scaler])

y_pred=clf.predict(X_test_pp)
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)

print(f"Model accuracy: {accuracy}")

Model accuracy: 0.9534632034632035


# Results

In [170]:
y_proba = clf.predict_proba(X_test_pp)

backtest_array = np.concatenate([pl.DataFrame(y_proba), np.reshape(y_test, (-1, 1))], axis=1)
backtest_pl = pl.DataFrame(backtest_array, schema=["failure", "success", "ground_truth"])
backtest_arrow = backtest_pl.to_arrow()

results = duckdb.query("""
select 
  t2.*,
  case when decile is not null
      then round(sum(conversions) over(order by decile desc nulls last rows between unbounded preceding and 0 preceding) / 
                  sum(lead_cnt) over(order by decile desc nulls last rows between unbounded preceding and 0 preceding), 4) 
      else 0.3912 end as precision,
  case when decile is not null
      then round(sum(conversions) over(order by decile desc nulls last rows between unbounded preceding and 0 preceding) / sum(conversions) over(), 4) 
      else 1 end as recall
from (
    select 
      decile,
      round(min(success), 4) as decile_value,
      count() as lead_cnt,
      sum(ground_truth) as conversions,
      round(avg(ground_truth), 4) as hit_rate
    from (select
            ntile(10) over(order by success asc) - 1 as decile,
            success,
            ground_truth
          from backtest_arrow
        ) t1
    group by grouping sets ((decile), ())
) t2
    order by decile desc nulls last
""")

print(results)

┌────────┬──────────────┬──────────┬─────────────┬──────────┬───────────┬────────┐
│ decile │ decile_value │ lead_cnt │ conversions │ hit_rate │ precision │ recall │
│ int64  │    double    │  int64   │   double    │  double  │  double   │ double │
├────────┼──────────────┼──────────┼─────────────┼──────────┼───────────┼────────┤
│      9 │       0.9959 │      184 │       182.0 │   0.9891 │    0.9891 │ 0.1259 │
│      8 │       0.9886 │      184 │       184.0 │      1.0 │    0.9946 │ 0.2531 │
│      7 │       0.9433 │      185 │       177.0 │   0.9568 │    0.9819 │ 0.3755 │
│      6 │       0.3466 │      185 │       143.0 │    0.773 │    0.9295 │ 0.4744 │
│      5 │       0.0484 │      185 │        22.0 │   0.1189 │    0.7671 │ 0.4896 │
│      4 │       0.0183 │      185 │        10.0 │   0.0541 │     0.648 │ 0.4965 │
│      3 │       0.0087 │      185 │         3.0 │   0.0162 │    0.5576 │ 0.4986 │
│      2 │       0.0038 │      185 │         1.0 │   0.0054 │    0.4885 │ 0.4993 │
│   